## Extract Poses from Amass Dataset

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib notebook
%matplotlib inline

import sys, os
import torch
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from tqdm import tqdm



from human_body_prior.tools.omni_tools import copy2cpu as c2c

os.environ['PYOPENGL_PLATFORM'] = 'egl'

/home/user/miniconda3/envs/PriorMDM/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Please remember to download the following subdataset from AMASS website: https://amass.is.tue.mpg.de/download.php. Note only download the <u>SMPL+H G</u> data.
* ACCD (ACCD)
* HDM05 (MPI_HDM05)
* TCDHands (TCD_handMocap)
* SFU (SFU)
* BMLmovi (BMLmovi)
* CMU (CMU)
* Mosh (MPI_mosh)
* EKUT (EKUT)
* KIT  (KIT)
* Eyes_Janpan_Dataset (Eyes_Janpan_Dataset)
* BMLhandball (BMLhandball)
* Transitions (Transitions_mocap)
* PosePrior (MPI_Limits)
* HumanEva (HumanEva)
* SSM (SSM_synced)
* DFaust (DFaust_67)
* TotalCapture (TotalCapture)
* BMLrub (BioMotionLab_NTroje)

### Unzip all datasets. In the bracket we give the name of the unzipped file folder. Please correct yours to the given names if they are not the same.

### Place all files under the directory **./amass_data/**. The directory structure shoud look like the following:  
./amass_data/  
./amass_data/ACCAD/  
./amass_data/BioMotionLab_NTroje/  
./amass_data/BMLhandball/  
./amass_data/BMLmovi/   
./amass_data/CMU/  
./amass_data/DFaust_67/  
./amass_data/EKUT/  
./amass_data/Eyes_Japan_Dataset/  
./amass_data/HumanEva/  
./amass_data/KIT/  
./amass_data/MPI_HDM05/  
./amass_data/MPI_Limits/  
./amass_data/MPI_mosh/  
./amass_data/SFU/  
./amass_data/SSM_synced/  
./amass_data/TCD_handMocap/  
./amass_data/TotalCapture/  
./amass_data/Transitions_mocap/  

**Please make sure the file path are correct, otherwise it can not succeed.**

In [2]:
# Choose the device to run the body model on.
comp_device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
from my_smpl import MySMPL

body_model = MySMPL("body_models/smpl", gender="neutral", ext="pkl").to(comp_device)


In [4]:
import pickle
input_dir='dataset/inter-human/motions/'
output_dir='dataset/inter-human/new_joints/'
os.makedirs(output_dir,exist_ok=True)
a=pickle.load(open(input_dir+"395.pkl",'rb'))
print(a['person1']["root_orient"].shape)

(49, 3)


In [16]:
ex_fps = 20
def amass_to_pose(src_path, save_path):
    with open(src_path,'rb') as f:
        data = pickle.load(f)
    fps = 0
    try:
        fps = data['mocap_framerate']
        frame_number = data["frames"]
        assert data['person1']["root_orient"].shape[0]/fps>0.5
    except:
#         print(list(bdata.keys()))
        return fps
    down_sample = round(fps / ex_fps)
    out={}
    with torch.no_grad():
        for person in ["person1","person2"]:
            bdata=data[person]
            root_orient = torch.Tensor(bdata['root_orient'][::down_sample]).to(comp_device) # controls the global root orientation
            pose_body=bdata['pose_body'][::down_sample]
            pose_body=np.concatenate((pose_body,np.zeros_like(pose_body[:,:6])),axis=1)
            pose_body = torch.Tensor(pose_body).to(comp_device) # controls the body
            betas = torch.Tensor(bdata['betas'][:10][np.newaxis]).to(comp_device) # controls the body shape
            trans = torch.Tensor(bdata['trans'][::down_sample]).to(comp_device)    
            body = body_model(body_pose=pose_body, betas=betas, global_orient=root_orient,transl=trans)
            out[person] = body.joints[:,:22,[0,2,1]].cpu().numpy()
    with open(save_path,'wb') as f:
        pickle.dump(out,f)
    return fps

amass_to_pose(input_dir+"1935.pkl",output_dir+"1935.pkl")

59.94

This will take a few hours for all datasets, here we take one dataset as an example

To accelerate the process, you could run multiple scripts like this at one time.

In [17]:
for path in tqdm(os.listdir(input_dir)):
    amass_to_pose(input_dir+path, output_dir+path)

100%|██████████████████████████████████████| 7810/7810 [01:14<00:00, 105.13it/s]


The above code will extract poses from **AMASS** dataset, and put them under directory **"./pose_data"**

The source data from **HumanAct12** is already included in **"./pose_data"** in this repository. You need to **unzip** it right in this folder.

## Segment, Mirror and Relocate Motions